## Implementation of Breadth First Search, Depth First Search

In [ ]:
def BFS_pathExists(grid,row,col,start,end):
      queue = []
      visited = {} #key = coordinate-tuples, value = previous location
      visited[(0, 0)] = (-1, -1)
      queue.append(start)
      goal = end
      while len(queue) > 0:
          r, c = queue.pop(0)
          if r == goal[0] and c == goal[1]:
              # build path backwards through the visited information
              path = []
              while r != -1:
                  path.append((r, c))
                  r, c = visited[(r, c)]
              path.reverse()
              #path = [list[i] for i in path]
              return path
          # find neighbours
          for dx, dy in ((-1, 0), (0, -1), (1, 0), (0, 1)):
              r_move = r + dy
              c_move = c + dx
              # check if move is valid
              if ( r_move >= 0 and r_move < row and c_move >= 0 and c_move < col and not (r_move, c_move) in visited and grid[r_move][c_move] != -1):
                  visited[(r_move, c_move)] = (r, c)
                  queue.append((r_move, c_move))
      return False

In [ ]:
def DFS_pathExists(grid,row,col,start,end):
      stack = []
      visited = {} #key = coordinate-tuples, value = previous location
      visited[tuple(start)] = (-1, -1)
      stack.append(tuple(start))
      goal = end
      while len(stack) > 0:
          r, c = stack.pop()
          if r == goal[0] and c == goal[1]:
              # build path backwards through the visited information
              path = []
              while r != -1:
                  path.append((r, c))
                  r, c = visited[(r, c)]
              path.reverse()
              return path
          # find neighbours
          d1 = (-1, 0), (0, -1), (1, 0), (0, 1)
          d2 = (1, 0), (0, 1),(-1, 0), (0, -1)
          for dx, dy in (d2):
              r_move = r + dy
              c_move = c + dx
              # check if move is valid
              if ( r_move >= 0 and r_move < row and c_move >= 0 and c_move < col and not (r_move, c_move) in visited and grid[r_move][c_move] != -1):
                  visited[(r_move, c_move)] = (r, c)
                  stack.append((r_move, c_move))
      return False

## Creation of Grid

In [ ]:
import numpy as np
import random
from random import choices
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from numba import jit, cuda
from timeit import default_timer as timer  


def visualizeGrid(data,n):
  cmap = ListedColormap(['red', 'blue'], 'indexed')
  #plt.imshow(data, cmap=plt.cm.RdBu)
  fig, ax = plt.subplots(figsize=(10,10))
  ax.scatter(0,0, marker = "*", color = "yellow", s = 200)
  ax.scatter(n-1,n-1, marker = "*", color = "green", s = 200)
  plt.imshow(data, cmap=cmap)

def visualizeGrid2(data,n):
  cmap = ListedColormap(['red', 'blue','white'], 'indexed')
  #plt.imshow(data, cmap=plt.cm.RdBu)
  fig, ax = plt.subplots(figsize=(10,10))
  ax.scatter(0,0, marker = "*", color = "yellow", s = 200)
  ax.scatter(n-1,n-1, marker = "*", color = "green", s = 200)
  plt.imshow(data, cmap=cmap)
  
def isBlock():
  # 0 = unblock, -1 = block
  states = [0,-1]
  states_prob = [0.72,0.28]
  final = choices(states, states_prob)
  return final[0]


def createGrid(n):
  start = [0,0]
  end = [n-1, n-1]
  flag = 0
  attempt = 0
  attempt_state = "Created grid is not valid, path between start and end doesn't exist"
  while flag==0:

    attempt += 1
    #creating empty grid of 51X51
    grid = np.empty((n, n), int)

    #adding blocks to grid
    blocked_cells = []
    rows = grid.shape[0]
    cols = grid.shape[1]
    for i in range(0,rows):
      for j in range(0,cols):
            grid[i][j] = isBlock()
            if grid[i][j]==-1:
              blocked_cells.append([i,j])

    
    #unblocking upper left and lower right corners
    grid[0][0] = 0
    grid[n-1,n-1] = 0


    #check if path exists from upper left cell to lower right cell
    begin = timer() 
    if DFS_pathExists(grid,n,n,start,end):
      flag = 1
      attempt_state = "Created grid is valid"
    
    #print("attempt {} : {}, verified in {} seconds.".format(attempt, attempt_state,timer()-begin))

  return grid, blocked_cells








## Modelling Ghosts and it's moves into maze


In [ ]:
import random

def create_ghosts(grid,n,n_ghosts):
  # finding all reachable nodes from start in the grid where we can place the ghosts
  # takes input - grid, dim(grid), #ghosts
  reachable_r = []
  reachable_c = []
  end = [n-1,n-1]
  for r in range(0,n):
    for c in range(0,n):
      start = [r,c] 
      if start!= end and start!= [0,0] and grid[r][c]!=-1:
        if DFS_pathExists(grid,n,n,start,end):
          reachable_r.append(r)
          reachable_c.append(c)
  
  # placing the ghosts in the grid in the reachable positions
  ghost_pos = []
  grid2 = grid.copy()
  for i in range(0,n_ghosts):
    t = random.randint(0, len(reachable_r)-1)
    t_r = reachable_r[t]
    t_c = reachable_c[t]
    grid2[t_r][t_c] = 2
    ghost_pos.append([t_r,t_c])

  return grid2, ghost_pos

def move_ghosts(grid,n,n_ghosts,ghost_pos):
  # takes input - grid, dim(grid), #ghosts, current ghost positions
  grid2 = grid.copy()
  direction = [[0,1],[0,-1],[1,0],[-1,0]]
  ghost_pos_new = []

  # for each ghost, find valid places to move, choose a place, and act accordingly to block/unblock cell condition
  for k in range(0,len(ghost_pos)):
    neighbours = []
    for i in range(0,len(direction)):
      r_move = ghost_pos[k][0] + direction[i][0]
      c_move = ghost_pos[k][1] + direction[i][1]
      if (r_move>=0 and r_move<n and c_move>=0 and c_move<n):
        neighbours.append([r_move,c_move])
    
    #print("neighbours: {}".format(neighbours))
    
    # choosing one of the neighbour randomly

    t = random.randint(0, len(neighbours)-1)
    r = neighbours[t][0]
    c = neighbours[t][1]

    if grid[r][c]==0:
      # if chosen cell is unblocked
      grid2[r][c] = 2
      grid2[ghost_pos[k][0]][ghost_pos[k][1]] = grid[ghost_pos[k][0]][ghost_pos[k][1]]
      ghost_pos_new.append([r,c])
    else:
      # if chosen cell is blocked
      if (random.randint(0,100) < 51):
        # move to new cell
        grid2[r][c] = 2
        grid2[ghost_pos[k][0]][ghost_pos[k][1]] = grid[ghost_pos[k][0]][ghost_pos[k][1]]
        ghost_pos_new.append([r,c])
      else:
        # remain in the same cell
        grid2[ghost_pos[k][0]][ghost_pos[k][1]] = 2
        ghost_pos_new.append([ghost_pos[k][0],ghost_pos[k][1]])

  return grid2, ghost_pos_new 
      

      



## Visualization

### Creation and saving snapshot of the maze at every timestep

In [ ]:
#create a2 folder first
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def getImage(path, zoom=0.04):
    return OffsetImage(plt.imread(path), zoom=zoom)
paths1 = ['/content/ghost1.png']
paths2 = ['/content/agent.jpeg']

def visualizeGrid_gif(data,n,path,ghost_pos,agent_pos,k,po):
  # for the creation of grid
  cmap = ListedColormap(['black', 'grey'], 'indexed')
  fig, ax = plt.subplots(figsize=(10,10))
  ax.scatter(0,0, marker = "*", color = "red", s = 200)
  ax.scatter(n-1,n-1, marker = "*", color = "red", s = 200)
  plt.imshow(data, cmap=cmap)
  # path
  x_coords=[a[0] for a in path]
  y_coords=[a[1] for a in path]
  ax.plot(y_coords,x_coords, color = "yellow")

  # org path
  x_coords=[a[0] for a in po]
  y_coords=[a[1] for a in po]
  ax.plot(y_coords,x_coords, color = "white")

  # ghosts
  x1_coords=[a[0] for a in ghost_pos]
  y1_coords=[a[1] for a in ghost_pos]
  for x0, y0 in zip(x1_coords, y1_coords):
    ab = AnnotationBbox(getImage(paths1[0]), (x0, y0), frameon=False)
    ax.add_artist(ab)
  
  # agent
  x2_coords=[a[0] for a in agent_pos]
  y2_coords=[a[1] for a in agent_pos]
  for x0, y0 in zip(y2_coords, x2_coords):
    bc = AnnotationBbox(getImage(paths2[0]), (x0, y0), frameon=False)
    ax.add_artist(bc)
  plt.savefig('/content/a2/{}.png'.format(k))
  plt.close(fig)

  return
  



### Creation of GIF from save snapshots

In [ ]:
import os
import imageio

# defining the path
png_dir = '/content/a2/'
images = []
temp = []
for i in range(1,38):
  temp.append(str(i)+'.png')

for file_name in temp:
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        print(file_path)
        images.append(imageio.imread(file_path))

# set the time for the the gif
for _ in range(10):
    images.append(imageio.imread(file_path))

imageio.mimsave('/content/a2/movie-1.gif', images,fps=55,duration=1)

In [ ]:
"""
import shutil,os

shutil.rmtree('/content/a2/')
os.mkdir('/content/a2/')
"""

### Visualization for Survivability vs Ghosts 

In [ ]:
import matplotlib.pyplot as plt

# creating a graph to visualize the performance of the agent for varying number of ghosts
def graph1():
  ghosts = []
  n = 51
  survivability_rates = []
  g = 0
  start = [0,0]
  end = [n-1,n-1]
  simulations = 10
  g_max = int((n*n)*0.15)

  while (g<=g_max):
    wins = 0
    for i in range(0,simulations):
      grid, blocked_cells = createGrid(n)
      grid2, ghost_pos = create_ghosts(grid,n,g)
      path,ghost_pos,state = agent1(grid, blocked_cells,g,ghost_pos,start,end)
      #path,ghost_pos,state = agent4(grid, blocked_cells,g,ghost_pos,start,end)
      if state=='agent wins!':
        wins = wins +1
    survivability = (wins/simulations) * 100
    ghosts.append(g)
    survivability_rates.append(survivability)
    #print('g: {} survival: {} g_max: {}'.format(g,survivability,g_max))
    g = g + 5

  # visualization
  x = ghosts
  y = survivability_rates
  plt.plot(x, y)
  plt.xticks(np.arange(min(x), max(x)+1, 10.0))
  plt.xlabel('Number of Ghosts')
  plt.ylabel('Agent 1 Survivability Rate')
  plt.title('Agent 1: Ghosts vs Survivability Rate')
  plt.savefig('Agent1_Survivability.png')
  plt.show()


  return ghosts, survivability_rates


ghosts,survivability_rates = graph1()
      




In [ ]:
# saving the data about each agent in a file
import pandas as pd
dict = {'Num_of_Ghosts':ghosts,'Survivability_Rates':survivability_rates}
df = pd.DataFrame(dict)
df.to_csv('agent1.csv')

In [ ]:
# visualization
  x = ghosts
  y = survivability_rates
  plt.plot(x, y)
  plt.xticks(np.arange(min(x), max(x)+1, 50.0))
  plt.xlabel('Number of Ghosts')
  plt.ylabel('Agent 1 Survivability Rate')
  plt.title('Agent 1: Ghosts vs Survivability Rate')
  plt.savefig('Agent1_Survivability.png')
  plt.show()

### Compare Function

In [ ]:
def compare(n,n_ghost,simulations):
  success_rate_a1 = 0
  success_rate_a2 = 0
  success_rate_a4 = 0
  for i in range(0,simulations):
    grid, blocked_cells = createGrid(n)
    grid2, ghost_pos = create_ghosts(grid,n,n_ghost)
    n_ghost = n_ghost
    start = [0,0]
    end = [grid.shape[0]-1,grid.shape[0]-1]
    path,ghost_pos,state1 = agent1(grid, blocked_cells,ghost_n,ghost_pos,start,end)
    path,ghost_pos,state2 = agent2(grid, blocked_cells,ghost_n,ghost_pos,start,end)
    path,ghost_pos,state4 = agent4(grid, blocked_cells,ghost_n,ghost_pos,start,end)
    if state1 == "agent wins!":
      success_rate_a1 += 1
    if state2 == "agent wins!":
      success_rate_a2 += 1
    if state4 == "agent wins!":
      success_rate_a4 += 1
  return (success_rate_a1/simulations)*100, (success_rate_a2/simulations)*100, (success_rate_a4/simulations)*100

n = 51
a1_s = []
a2_s = []
a4_s = []
ghosts = []
i = 1
while (i <= (int(0.1*n*n))):
  ghosts.append(i)
  n_ghost = i
  simulations = 20
  success_rate_a1, success_rate_a2,success_rate_a4 = compare(n,n_ghost,simulations)
  a1_s.append(success_rate_a1)
  a2_s.append(success_rate_a2)
  a4_s.append(success_rate_a4)
  #print("agent 1 : {}, agent 2 : {}, agent 4 : {}".format(success_rate_a1, success_rate_a2,success_rate_a4))
  i = i+5

x = ghosts
plt.plot(x, a1_s, 'r') # plotting t, a separately 
plt.plot(x, a2_s, 'b')
plt.plot(x, a4_s, 'g')
plt.xlabel('Number of Ghosts')
plt.ylabel('Agent Survivability Rate')
plt.title('Agent: Ghosts vs Survivability Rate')
plt.savefig('graph1.png')
plt.show()

## Agent 1

Agent 1)  Agent  1  plans  a  the  shortest  path  through  the  maze  and  executes  it,  ignoring  the  ghosts.   This  agent  isincredibly efficient - it only has to plan a path once - but it makes no adjustments or updates due to a changingenvironment.

In [ ]:
def agent1(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  n = grid.shape[0]
  start = start
  end = end
  planned_path = BFS_pathExists(grid,n,n,start,end)
  #print(planned_path)
  k = 1
  for i in range(0,len(planned_path)):
    t = [list(planned_path[i])]
    #visualizeGrid_gif(grid,n,path,ghost_pos,t,k)  --> create a2 folder, and uncomment this when you want the animation
    k += 1
    if [planned_path[i][0],planned_path[i][1]] in ghost_pos:
      return planned_path,ghost_pos,"agent dead!"
    x , ghost_pos = move_ghosts(grid,n,ghost_n,ghost_pos)
  return planned_path,ghost_pos,"agent wins!"



  


In [ ]:
n = 51
grid, blocked_cells = createGrid(n)
ghost_n = 15
grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
start = [0,0]
end = [grid.shape[0]-1,grid.shape[0]-1]
begin = timer() 
x,ghost_pos, b = agent1(grid, blocked_cells,ghost_n,ghost_pos,start,end)
print(timer()-begin,b, x)

0.010280339000019012 agent wins! [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (2, 6), (3, 6), (4, 6), (4, 7), (5, 7), (5, 8), (6, 8), (6, 9), (6, 10), (6, 11), (7, 11), (8, 11), (9, 11), (10, 11), (11, 11), (12, 11), (13, 11), (14, 11), (14, 12), (15, 12), (16, 12), (17, 12), (18, 12), (18, 13), (19, 13), (20, 13), (21, 13), (22, 13), (23, 13), (23, 14), (24, 14), (25, 14), (25, 15), (25, 16), (25, 17), (25, 18), (25, 19), (25, 20), (25, 21), (26, 21), (27, 21), (27, 22), (27, 23), (27, 24), (27, 25), (27, 26), (27, 27), (28, 27), (29, 27), (29, 28), (29, 29), (29, 30), (29, 31), (29, 32), (30, 32), (31, 32), (31, 33), (31, 34), (32, 34), (33, 34), (34, 34), (34, 35), (34, 36), (35, 36), (36, 36), (37, 36), (38, 36), (39, 36), (40, 36), (41, 36), (41, 37), (41, 38), (41, 39), (42, 39), (42, 40), (42, 41), (43, 41), (44, 41), (45, 41), (45, 42), (45, 43), (45, 44), (46, 44), (47, 44), (47, 45), (47, 46), (47, 47), (48, 47), (48, 48), (49, 48), (49, 49), (49, 50), (50

## Agent 2

### A* Implementation

In [ ]:
import math
import numpy as np
import heapq
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# to estimate path to the goal node
def manhattan(a, b):
  return abs(b[0]-a[0]) + abs(b[1] - a[1])
 
def astar(array, start, goal):
  neighbors = [(0,1),(0,-1),(1,0),(-1,0)]
  closed_set = set()
  parent = {}
  g_score = {start:0}
  f_score = {start:manhattan(start, goal)}
  # open list 
  h = []
  heapq.heappush(h, (f_score[start], start))
 
 # check for all available positions
  while h:
    # neighbour with least f score
    curr = heapq.heappop(h)[1]

    # if goal is reached, return path
    if curr == goal:
      data = []

      while curr in parent:
        data.append(curr)
        curr = parent[curr]
      paths = data + [start]
      paths = paths[::-1]
      return paths
    closed_set.add(curr)

    # compute g_scores of neighbours
    for i, j in neighbors:
      neighbor = curr[0] + i, curr[1] + j
      t_gs = g_score[curr] + manhattan(curr, neighbor)

      # ignore unvalid neighbors
      if 0 <= neighbor[0] < array.shape[0]:
        if 0 <= neighbor[1] < array.shape[1]:                
          if array[neighbor[0]][neighbor[1]] == -1 :
            continue
          else:
            continue
      else:
        continue
      # if neighbour closed set, and G score> G score of pos position, ignore 
      if neighbor in closed_set and t_gs >= g_score.get(neighbor, 0):
        continue
      # if G score is less and neighbour not explored
      if  t_gs < g_score.get(neighbor, 0) or neighbor not in [i[1]for i in h]:
        parent[neighbor] = curr
        g_score[neighbor] = t_gs
        f_score[neighbor] = t_gs + manhattan(neighbor, goal)
        heapq.heappush(h, (f_score[neighbor], neighbor))
    
    return False

### Agent 2 Function

In [ ]:

def a1(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  n = grid.shape[0]
  grid2 = grid.copy()
  for i in ghost_pos:
    grid2[i[0]][i[1]] = -1
  planned_path = astar(np.array(grid),tuple(start),tuple(end))
  return planned_path 

def most_common(a):
    return max(set(a), key=a.count)

def manhattan(a, b):
  return abs(b[0]-a[0]) + abs(b[1] - a[1])

def find_next_move(grid, blocked_cells,ghost_n,ghost_pos,curr):
  direction = [[0,1],[0,-1],[1,0],[-1,0]]
  neighbours = [curr]
  n = grid.shape[0]

  #find neighbours
  for i in range(len(direction)):
    r_move = curr[0]+direction[i][0]
    c_move = curr[1]+direction[i][1]
    if (r_move>=0 and r_move<n and c_move>=0 and c_move<n and [r_move,c_move] not in ghost_pos ):
        neighbours.append([r_move,c_move])
    
  # finding nearest ghosts
  if len(neighbours)>0:
    ghost_distances = []
    for i in range(len(ghost_pos)):
      ghost_distances.append(manhattan(curr,ghost_pos[i]))
    min_ghost_distance = min(ghost_distances)
    nearest_ghosts = []
    for j in range(0,len(ghost_distances)):
      if ghost_distances[j] == min_ghost_distance:
        nearest_ghosts.append(ghost_pos[j])
    #print(nearest_ghosts)

    # finding what move to take next
    moves = []
    if len(nearest_ghosts)>0:
      for i in range(len(nearest_ghosts)):
        x = []
        for j in range(len(neighbours)):
          x.append(manhattan(neighbours[j],nearest_ghosts[i]))
        maxIndex, maxValue = max(enumerate(x), key=lambda v: v[1])
        moves.append(tuple(neighbours[maxIndex]))
      #print(moves)
      return list(most_common(moves))
    return curr

def agent2(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  curr_path = []
  n = grid.shape[0]
  curr = start
  curr_path.append(curr)
  i = 0
  k = 1
  planned_path = a1(grid, blocked_cells,ghost_n,ghost_pos,curr,end)
  #print(planned_path)

  if (planned_path):
    planned_path.pop(0)
  while(curr!=end):
    #print(curr)
    #visualizeGrid_gif(grid,n,curr_path,ghost_pos,[curr],k,po)
    k += 1
    if list(curr) in ghost_pos:
      return curr_path, ghost_pos, "agent dead!"

    #re-route only when the ghosts appear in the current 4 moves of the planned path
    if(planned_path and len(planned_path)>3):
      if ((list(planned_path[0]) in ghost_pos) or (list(planned_path[1]) in ghost_pos) or (list(planned_path[2]) in ghost_pos)):
        #print(curr)
        #print("old path:",planned_path)

        #make the planned_path[0] as blocked, send it to the path search function
        temp = grid.copy()

        #print(planned_path[0][0],planned_path[0][1])
        temp[planned_path[0][0],planned_path[0][1]] = -1
        planned_path = a1(temp, blocked_cells,ghost_n,ghost_pos,list(curr),end)
        if(planned_path):
          planned_path.pop(0)
        #print("new path:",planned_path)
    
        
    if (planned_path and (len(planned_path)>0)):
      #print(list(planned_path[1]))

      curr = list(planned_path[0])
      planned_path.pop(0)
      curr_path.append(curr)
    else:
      #move away from the nearest ghost if possible or stay in place
      curr = find_next_move(grid, blocked_cells,ghost_n,ghost_pos,curr)
      curr_path.append(curr)
    #print(curr_path)
    x , ghost_pos = move_ghosts(grid,n,ghost_n,ghost_pos)
  return  curr_path,ghost_pos, "agent wins!"



In [ ]:
  n = 20
  grid, blocked_cells = createGrid(n)
  ghost_n = 5
  grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
  start = [0,0]
  end = [grid.shape[0]-1,grid.shape[0]-1]
  begin = timer() 
  x,ghost_pos, b = agent2(grid, blocked_cells,ghost_n,ghost_pos,start,end)
  print(timer()-begin,b, x)





0.08263813700068567 agent wins! [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 18], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 19], [0, 1

In [ ]:
"""

i = 30
n = 20
while(i>0):
  grid, blocked_cells = createGrid(n)
  ghost_n = 5
  grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
  start = [0,0]
  end = [grid.shape[0]-1,grid.shape[0]-1]
  begin1 = timer() 
  x,ghost_pos, b = agent2(grid, blocked_cells,ghost_n,ghost_pos,start,end)
  begin2 = timer() 
  #x1,ghost_pos1, b1 = agent1(grid, blocked_cells,ghost_n,ghost_pos,start,end)
  print(timer()-begin1,b, x)
  #print(timer()-begin2,b1, x1)
  print()
  i -= 1

"""


## Agent 3

In [ ]:
import math
import numpy as np
import heapq
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


def manhattan(a, b):
  return abs(b[0]-a[0]) + abs(b[1] - a[1])



In [ ]:
def compute_success_rate(grid, blocked_cells,ghost_n, ghost_pos, start, end):
  simulations = 50
  total_wins = 0
  #early_termination = 4

  for i in range(0,simulations):
    x, ghost_pos, b = agent2(grid, blocked_cells,ghost_n,ghost_pos,start,end)
    if b == 'agent wins!':
      total_wins = total_wins + 1
  return (total_wins/simulations)*100



In [ ]:
def agent3(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  curr = start
  direction = [[0,1],[0,-1],[1,0],[-1,0]]
  path = []
  path.append(curr)
  while(curr!=end):

    #ghosts make a move
    x , ghost_pos = move_ghosts(grid,n,ghost_n,ghost_pos)

    # condition for agent's failure
    if curr in ghost_pos:
      return path,ghost_pos, "agent dead!"
    
    
    # finding valid moves for agent 
    neighbours = []
    for i in range(0,len(direction)):
      r_move = curr[0] + direction[i][0]
      c_move = curr[1] + direction[i][1]
      if (r_move>=0 and r_move<n and c_move>=0 and c_move<n and grid[r_move,c_move]!=-1 and [r_move,c_move] not in ghost_pos):
        #if a(grid, blocked_cells,ghost_n,ghost_pos,[r_move,c_move],end):
        neighbours.append([r_move,c_move])
    
    # add curr to neighbours -- new
    neighbours.append(curr)

    # finding success rate for the valid moves
    success_rate = []
    if len(neighbours)!=0:
      for i in range(0,len(neighbours)):
        k = neighbours[i]
        #print(compute_success_rate(grid, blocked_cells,ghost_n, ghost_pos, k, end))
        success_rate.append(compute_success_rate(grid, blocked_cells,ghost_n, ghost_pos, k, end))
      
      print("success rates:",success_rate)
      
      # choosing move with the best success rate
      max_sr = max(success_rate)
      temp = [] # max success rates
      temp2 = []
      dist = []
      for i in range(0,len(success_rate)):
        if success_rate[i] == max_sr:
          temp.append(i)  #indices with max success rate
          dist.append(manhattan(neighbours[i], end))
      # finding the best move (least distance from goal) among multiple neighbours with best success rate
      min_dist = min(dist)
      for i in range(0,len(temp)):
        if dist[i] == min_dist:
          temp2.append(i)  #indices with max success rate and min dist to goal
  

      if len(temp2)>0:
        r = random.randint(0,len(temp2)-1)


      #agent makes a move to cell with best chances
      curr = neighbours[r]
      path.append(curr)

  return path,ghost_pos,"agent wins"


for i in range(0,2):
  n = 10
  grid, blocked_cells = createGrid(n)
  ghost_n = 1
  grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
  start = [0,0]
  end = [grid.shape[0]-1,grid.shape[0]-1]
  begin = timer() 
  x,ghost_pos,b = agent3(grid, blocked_cells,ghost_n,ghost_pos,start,end)
  print(b, timer()-begin, x)


  


## Agent 4

In [ ]:
import math
import numpy as np
import heapq
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



def a4(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  n = grid.shape[0]
  grid2 = grid.copy()
  for i in ghost_pos:
    grid2[i[0]][i[1]] = -1

  planned_path = astar(np.array(grid),tuple(start),tuple(end))
  return planned_path 


def manhattan(a, b):
  return abs(b[0]-a[0]) + abs(b[1] - a[1])

def ghost_nearby(grid, ghost_pos,node):
  for i in range(0,len(ghost_pos)):
    if manhattan(ghost_pos[i],node) == 1:
      return True
  return False

def reroute(grid, blocked_cells,ghost_n,ghost_pos,start,end, planned_path):
  if (planned_path and len(planned_path)>3 and ((list(planned_path[0]) in ghost_pos) or (list(planned_path[1]) in ghost_pos) or (list(planned_path[2]) in ghost_pos))):
    return True
  elif (planned_path and len(planned_path)>2 and (ghost_nearby(grid, ghost_pos,list(planned_path[0])) or ghost_nearby(grid, ghost_pos,list(planned_path[1])) )):
    return True
  else:
    return False
    #print("new path:",planned_path)

def agent4(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  curr_path = []
  n = grid.shape[0]
  curr = start
  curr_path.append(curr)
  i = 0
  k = 1
  planned_path = a1(grid, blocked_cells,ghost_n,ghost_pos,curr,end)
  po = planned_path
  if (planned_path):
    planned_path.pop(0)
  while(curr!=end):
    #print(curr)
    #visualizeGrid_gif(grid,n,curr_path,ghost_pos,[curr],k,po)
    k += 1
    if list(curr) in ghost_pos:
      return curr_path, ghost_pos, "agent dead!"

  #re-route only when the ghosts appear in the next 3 moves of the planned path or near the neighbours of the next 3 moves
    if(reroute(grid, blocked_cells,ghost_n,ghost_pos,start,end, planned_path)):
      po = planned_path
      #print("old path",planned_path)
      #make the planned_path[0] as blocked, send it to the path search function
      temp = grid.copy()
      #print(planned_path[0][0],planned_path[0][1])
      temp[planned_path[0][0],planned_path[0][1]] = -1
      planned_path = a4(temp, blocked_cells,ghost_n,ghost_pos,list(curr),end)
      if(planned_path):
        planned_path.pop(0)
      #print("new path:",planned_path)
    
    #print(planned_path)
        
    if (planned_path and (len(planned_path)>0)):
      #print(list(planned_path[1]))
      curr = list(planned_path[0])
      planned_path.pop(0)
      curr_path.append(curr)
    else:
      #move away from the nearest ghost if possible or stay in place
      curr = find_next_move(grid, blocked_cells,ghost_n,ghost_pos,curr)
      curr_path.append(curr)
    #print(curr_path)
    x , ghost_pos = move_ghosts(grid,n,ghost_n,ghost_pos)
  return  curr_path,ghost_pos, "agent wins!"



In [ ]:
n = 51
grid, blocked_cells = createGrid(n)
ghost_n = 20
grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
start = [0,0]
end = [grid.shape[0]-1,grid.shape[0]-1]
begin = timer() 
x,ghost_pos,b = agent4(grid, blocked_cells,ghost_n,ghost_pos,start,end)
print(b, timer()-begin, x)

agent wins! 0.08224713999993583 [[0, 0], [0, 0], [0, 1], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 0], [2, 0], [2, 1], [2, 2], [2, 1], [2, 0], [2, 1], [2, 2], [2, 1], [2, 2], [2, 3], [2, 2], [2, 3], [2, 2], [2, 3], [2, 2], [2, 1], [2, 0], [1, 0], [2, 0], [3, 0], [2, 0], [3, 0], [2, 0], [3, 0], [4, 0], [3, 0], [4, 0], [3, 0], [2, 0], [3, 0], [2, 0], [1, 0], [2, 0], [3, 0], [2, 0], [3, 0], [2, 0], [3, 0], [2, 0], [3, 0], [4, 0], [3, 0], [2, 0], [3, 0], [2, 0], [1, 0], [2, 0], [1, 0], [2, 0], [3, 0], [2, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [4, 0], [5, 0], [6, 0], [5, 0], [4, 0], [5, 0], [6, 0], [5, 0], [4, 0], [5, 0], [4, 0], [3, 0], [4, 0], [3, 0], [4, 0], [3, 0], [4, 0], [3, 0], [2, 0], [3, 0], [2, 0],

In [ ]:
for i in range(0,10):
  n = 51
  grid, blocked_cells = createGrid(n)
  ghost_n = 20
  grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
  start = [0,0]
  end = [grid.shape[0]-1,grid.shape[0]-1]
  begin = timer() 
  x,ghost_pos,b = agent4(grid, blocked_cells,ghost_n,ghost_pos,start,end)
  print(b, timer()-begin, x)

## Agent 5

In [ ]:
import math
import numpy as np
import heapq
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



def a5(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  n = grid.shape[0]
  grid2 = grid.copy()
  for i in ghost_pos:
    grid2[i[0]][i[1]] = -1

  planned_path = astar(np.array(grid),tuple(start),tuple(end))
  return planned_path 


def manhattan(a, b):
  return abs(b[0]-a[0]) + abs(b[1] - a[1])

def ghost_nearby(grid, ghost_pos,node):
  for i in range(0,len(ghost_pos)):
    if manhattan(ghost_pos[i],node) == 1:
      return True
  return False

def reroute(grid, blocked_cells,ghost_n,ghost_pos,start,end, planned_path):
  if (planned_path and len(planned_path)>3 and ((list(planned_path[0]) in ghost_pos) or (list(planned_path[1]) in ghost_pos) or (list(planned_path[2]) in ghost_pos))):
    return True
  elif (planned_path and len(planned_path)>2 and (ghost_nearby(grid, ghost_pos,list(planned_path[0])) or ghost_nearby(grid, ghost_pos,list(planned_path[1])) )):
    return True
  else:
    return False
    #print("new path:",planned_path)

def remove_ghosts_in_blocked_cell(ghost_pos, blocked_cells):
    ghosts_blocked=[]
    for i in ghost_pos:
        if i in blocked_cells:
            ghosts_blocked.append(ghost_pos.pop(ghost_pos.index(i)))
    return ghost_pos, ghosts_blocked

def add_ghosts_in_blocked_cell(ghost_pos, ghosts_blocked):
    for i in ghosts_blocked:
        ghost_pos.append(i)
    return ghost_pos


def agent5(grid, blocked_cells,ghost_n,ghost_pos,start,end):
  curr_path = []
  n = grid.shape[0]
  curr = start
  curr_path.append(curr)
  i = 0
  k = 1
  planned_path = a1(grid, blocked_cells,ghost_n,ghost_pos,curr,end)
  #print(planned_path)
  po = planned_path
  if (planned_path):
    planned_path.pop(0)
  while(curr!=end):

    #visualizeGrid_gif(grid,n,curr_path,ghost_pos,[curr],k,po)
    k += 1
    if list(curr) in ghost_pos:
      return curr_path, ghost_pos, "agent dead!"

    # make walled ghosts invisible to agent
    ghost_pos, ghosts_blocked = remove_ghosts_in_blocked_cell(ghost_pos, blocked_cells)

  #re-route only when the ghosts appear in the next 3 moves of the planned path or near the neighbours of the next 3 moves
    if(reroute(grid, blocked_cells,ghost_n,ghost_pos,start,end, planned_path)):
      po = planned_path
    
      #make the planned_path[0] as blocked, send it to the path search function
      temp = grid.copy()
      #print(planned_path[0][0],planned_path[0][1])
      temp[planned_path[0][0],planned_path[0][1]] = -1
      planned_path = a5(temp, blocked_cells,ghost_n,ghost_pos,list(curr),end)
      if(planned_path):
        planned_path.pop(0)
      #print("new path:",planned_path)
    
    #print(planned_path)
        
    if (planned_path and (len(planned_path)>0)):
      #print(list(planned_path[1]))
      curr = list(planned_path[0])
      planned_path.pop(0)
      curr_path.append(curr)
    else:
      #move away from the nearest ghost if possible or stay in place
      curr = find_next_move(grid, blocked_cells,ghost_n,ghost_pos,curr)
      curr_path.append(curr)
    #print(curr_path)
    ghost_pos = add_ghosts_in_blocked_cell(ghost_pos, ghosts_blocked)
    x , ghost_pos = move_ghosts(grid,n,ghost_n,ghost_pos)
  return  curr_path,ghost_pos, "agent wins!"

In [ ]:
n = 51
grid, blocked_cells = createGrid(n)
ghost_n = 20
grid2, ghost_pos = create_ghosts(grid,n,ghost_n)
start = [0,0]
end = [grid.shape[0]-1,grid.shape[0]-1]
begin = timer() 
x,ghost_pos,b = agent5(grid, blocked_cells,ghost_n,ghost_pos,start,end)
print(b, timer()-begin, x)

agent wins! 2.3388951619999716 [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [5, 0], [6, 0], [5, 0], [6, 0], [5, 0], [6, 0], [5, 0], [6, 0], [6, 1], [6, 2], [6, 3], [6, 2], [6, 3], [6, 4], [6, 3], [6, 2], [6, 3], [6, 2], [6, 1], [6, 2], [6, 3], [6, 2], [6, 3], [6, 4], [6, 3], [6, 2], [6, 3], [6, 2], [6, 3], [6, 2], [6, 3], [6, 2], [6, 3], [6, 4], [6, 3], [6, 4], [6, 5], [6, 4], [6, 5], [6, 4], [6, 5], [6, 6], [6, 5], [6, 4], [6, 5], [6, 4], [6, 5], [6, 4], [6, 5], 